In [1]:
# Import our dependencies
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import pandas as pd
import tensorflow as tf

data = Path('Resourses/patient_data.csv')
df = pd.read_csv(data)
df.head()

,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses,change,diabetesMed,...,glyburide-metformin:Up,A1Cresult:>7,A1Cresult:>8,A1Cresult:None,A1Cresult:Norm,max_glu_serum:>200,max_glu_serum:>300,max_glu_serum:None,max_glu_serum:Norm,readmitted
0,4,66,3,18,0,0,1,9,1,1,...,0,0,0,1,0,0,0,1,0,1
1,2,48,0,15,4,0,0,7,0,0,...,0,0,0,1,0,0,0,1,0,0
2,4,21,3,23,1,0,2,7,0,1,...,0,0,0,1,0,0,0,1,0,0
3,5,38,0,5,0,0,0,2,1,1,...,0,0,1,0,0,0,0,1,0,0
4,1,6,0,6,0,0,0,6,0,1,...,0,0,0,1,0,0,0,1,0,1


In [2]:
# Split target column from dataset
y = df['readmitted']
X = df.drop(columns='readmitted')

In [3]:
# Use sklearn to split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

# Create scaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [6]:
nn_model = tf.keras.models.Sequential()
nn_model.add(tf.keras.layers.Dense(units=16, activation="relu", input_dim=14))
nn_model.add(tf.keras.layers.Dense(units=16, activation="relu"))
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=50)

# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/50


ValueError: Exception encountered when calling Sequential.call().

[1mInput 0 of layer "dense_3" is incompatible with the layer: expected axis -1 of input shape to have value 14, but received input with shape (None, 150)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(None, 150), dtype=float32)
  • training=True
  • mask=None